# Chapter 7: 単語ベクトル

単語の意味を実ベクトルで表現する単語ベクトル（単語埋め込み）に関して，以下の処理を行うプログラムを作成せよ．

## 60. 単語ベクトルの読み込みと表示

Google Newsデータセット（約1,000億単語）での[学習済み単語ベクトル](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit)（300万単語・フレーズ，300次元）をダウンロードし，”United States”の単語ベクトルを表示せよ．ただし，”United States”は内部的には”United_States”と表現されていることに注意せよ．

In [11]:
!git clone https://github.com/chentinghao/download_google_drive.git ../data/download_google_drive

Cloning into '../data/download_google_drive'...
remote: Enumerating objects: 16, done.
remote: Total 16 (delta 0), reused 0 (delta 0), pack-reused 16
Unpacking objects: 100% (16/16), 4.77 KiB | 116.00 KiB/s, done.


In [16]:
!poetry run python ../data/download_google_drive/download_gdrive.py 0B7XkCwpI5KDYNlNUTTlSS21pQmM ../data/google_news.bin.gz

../data/download_google_drive/download_gdrive.py:50: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if len(sys.argv) is not 3:
1.53GB [00:32, 50.5MB/s]


In [15]:
from gensim.models import KeyedVectors

In [18]:
model = KeyedVectors.load_word2vec_format('../data/google_news.bin.gz', binary=True)

In [19]:
model['United_States']

array([-3.61328125e-02, -4.83398438e-02,  2.35351562e-01,  1.74804688e-01,
       -1.46484375e-01, -7.42187500e-02, -1.01562500e-01, -7.71484375e-02,
        1.09375000e-01, -5.71289062e-02, -1.48437500e-01, -6.00585938e-02,
        1.74804688e-01, -7.71484375e-02,  2.58789062e-02, -7.66601562e-02,
       -3.80859375e-02,  1.35742188e-01,  3.75976562e-02, -4.19921875e-02,
       -3.56445312e-02,  5.34667969e-02,  3.68118286e-04, -1.66992188e-01,
       -1.17187500e-01,  1.41601562e-01, -1.69921875e-01, -6.49414062e-02,
       -1.66992188e-01,  1.00585938e-01,  1.15722656e-01, -2.18750000e-01,
       -9.86328125e-02, -2.56347656e-02,  1.23046875e-01, -3.54003906e-02,
       -1.58203125e-01, -1.60156250e-01,  2.94189453e-02,  8.15429688e-02,
        6.88476562e-02,  1.87500000e-01,  6.49414062e-02,  1.15234375e-01,
       -2.27050781e-02,  3.32031250e-01, -3.27148438e-02,  1.77734375e-01,
       -2.08007812e-01,  4.54101562e-02, -1.23901367e-02,  1.19628906e-01,
        7.44628906e-03, -

## 61. 単語の類似度

“United States”と”U.S.”のコサイン類似度を計算せよ．

In [20]:
model.similarity('United_States', 'U.S.')

0.73107743

## 62. 類似度の高い単語10件

“United States”とコサイン類似度が高い10語と，その類似度を出力せよ．

In [22]:
model.most_similar('United_States', topn=10)

[('Unites_States', 0.7877248525619507),
 ('Untied_States', 0.7541370391845703),
 ('United_Sates', 0.74007248878479),
 ('U.S.', 0.7310774326324463),
 ('theUnited_States', 0.6404393911361694),
 ('America', 0.6178410053253174),
 ('UnitedStates', 0.6167312264442444),
 ('Europe', 0.6132988929748535),
 ('countries', 0.6044804453849792),
 ('Canada', 0.6019070148468018)]

## 63. 加法構成性によるアナロジー

“Spain”の単語ベクトルから”Madrid”のベクトルを引き，”Athens”のベクトルを足したベクトルを計算し，そのベクトルと類似度の高い10語とその類似度を出力せよ．

In [23]:
model.most_similar(positive=['Spain', 'Athens'], negative=['Madrid'], topn=10)

[('Greece', 0.6898481249809265),
 ('Aristeidis_Grigoriadis', 0.5606848001480103),
 ('Ioannis_Drymonakos', 0.5552908778190613),
 ('Greeks', 0.545068621635437),
 ('Ioannis_Christou', 0.5400862693786621),
 ('Hrysopiyi_Devetzi', 0.5248444676399231),
 ('Heraklio', 0.5207759737968445),
 ('Athens_Greece', 0.516880989074707),
 ('Lithuania', 0.5166866183280945),
 ('Iraklion', 0.5146791934967041)]

## 64. アナロジーデータでの実験

[単語アナロジーの評価データ](http://download.tensorflow.org/data/questions-words.txt)をダウンロードし，vec(2列目の単語) - vec(1列目の単語) + vec(3列目の単語)を計算し，そのベクトルと類似度が最も高い単語と，その類似度を求めよ．求めた単語と類似度は，各事例の末尾に追記せよ．

In [55]:
!wget "http://download.tensorflow.org/data/questions-words.txt" -P ../data/

--2020-11-03 18:24:07--  http://download.tensorflow.org/data/questions-words.txt
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.25.112, 2404:6800:4004:819::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.25.112|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 603955 (590K) [text/plain]
Saving to: ‘../data/questions-words.txt.2’

questions-words.txt 100%[===================>] 589.80K  --.-KB/s    in 0.02s   

2020-11-03 18:24:07 (38.1 MB/s) - ‘../data/questions-words.txt.2’ saved [603955/603955]



In [57]:
!head ../data/questions-words.txt

In [58]:
!sed --in-place '/^:/d' ../data/questions-words.txt

In [75]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

/mnt/Storage/Users/shu/.cache/pypoetry/virtualenvs/nlp100-st2962ze-py3.8/lib/python3.8/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [65]:
df = pd.read_csv('../data/questions-words.txt', sep=' ', header=None)

In [67]:
df.columns = ['one', 'two', 'three', 'four']

In [71]:
def infer(row):
    res = model.most_similar(positive=[row['two'], row['three']], negative=['one'], topn=1)
    return pd.Series([res[0][0], res[0][1]])

In [76]:
df[['five', 'six']] = df.progress_apply(infer, axis=1)

100%|██████████| 19544/19544 [42:47<00:00,  7.61it/s]


In [77]:
df

,one,two,three,four,five,six
0,Athens,Greece,Baghdad,Iraq,Iraqi,0.536203
1,Athens,Greece,Bangkok,Thailand,Thailand,0.533668
2,Athens,Greece,Beijing,China,Bejing,0.552319
3,Athens,Greece,Berlin,Germany,Athens,0.519207
4,Athens,Greece,Bern,Switzerland,Athens,0.474905
...,...,...,...,...,...,...
19539,write,writes,talk,talks,opines,0.469121
19540,write,writes,think,thinks,opines,0.597886
19541,write,writes,vanish,vanishes,disappear,0.485261
19542,write,writes,walk,walks,wrote,0.422914


In [78]:
df.to_csv('../data/questions-words-infered.txt', sep=' ', header=None, index=None)